In [1]:
import requests
from requests import Response

youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res:Response = requests.get(youbike_url)
if res.status_code == 200:
    print("下載成功")
else:
    print("下載失敗")


from pydantic import BaseModel, Field, RootModel, field_validator

class Site(BaseModel):
    站名:str = Field(alias = 'sna')
    所在區:str = Field(alias='sarea')
    位置:str = Field(alias='ar')
    日期:str = Field(alias='mday')
    狀態:bool = Field(alias = "act")
    總車輛數:int = Field(alias='total')
    可借:int = Field(alias='available_rent_bikes')
    可還:int = Field(alias='available_return_bikes')
    經度:float = Field(alias='longitude')
    緯度:float = Field(alias='latitude')

# 將驗證器附加到模型的特定欄位  若要先進行驗證再輸出 用 mode="before" 

    @field_validator("站名")
    @classmethod
    def abc(cls,value:str)->str:
        return value.split("_")[-1]

class YouBike(RootModel):
    root: list[Site]

data:YouBike = YouBike.model_validate_json(res.text)

all_sites:list = data.model_dump()
all_sites

import pandas as pd
df1 = pd.DataFrame(all_sites)
df1

下載成功


,站名,所在區,位置,日期,狀態,總車輛數,可借,可還,經度,緯度
0,捷運科技大樓站,大安區,復興南路二段235號前,2024-06-19 07:42:21,True,28,2,26,121.54360,25.02605
1,復興南路二段273號前,大安區,復興南路二段273號西側,2024-06-19 07:40:31,True,21,5,16,121.54357,25.02565
2,國北教大實小東側門,大安區,和平東路二段96巷7號,2024-06-19 07:35:22,True,16,14,2,121.54124,25.02429
3,和平公園東側,大安區,和平東路二段118巷33號,2024-06-19 07:34:20,True,11,3,8,121.54282,25.02351
4,辛亥復興路口西北側,大安區,復興南路二段368號,2024-06-19 07:42:21,True,16,14,2,121.54299,25.02153
...,...,...,...,...,...,...,...,...,...,...
1422,臺大總圖書館西南側,臺大公館校區,臺大圖書館西南側,2024-06-19 07:39:21,True,30,5,25,121.54031,25.01690
1423,臺大黑森林西側,臺大公館校區,臺大霖澤館南側,2024-06-19 07:39:21,True,20,2,18,121.54347,25.01995
1424,臺大獸醫館南側,臺大公館校區,臺大獸醫系館南側,2024-06-19 07:40:31,True,24,3,21,121.54242,25.01791
1425,臺大新體育館東南側,臺大公館校區,臺大體育館東側,2024-06-19 07:38:20,True,40,17,23,121.53591,25.02112


In [2]:
df1['可借'] <= 3

0        True
1       False
2       False
3        True
4       False
        ...  
1422    False
1423     True
1424     True
1425    False
1426    False
Name: 可借, Length: 1427, dtype: bool

pandas.DataFrame.info
- DataFrame.info(verbose=None, buf=None, max_cols=None, memory_usage=None, show_counts=None)[source]
- Print a concise summary of a DataFrame.

In [3]:
#檢查  null(空的)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1427 entries, 0 to 1426
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   站名      1427 non-null   object 
 1   所在區     1427 non-null   object 
 2   位置      1427 non-null   object 
 3   日期      1427 non-null   object 
 4   狀態      1427 non-null   bool   
 5   總車輛數    1427 non-null   int64  
 6   可借      1427 non-null   int64  
 7   可還      1427 non-null   int64  
 8   經度      1427 non-null   float64
 9   緯度      1427 non-null   float64
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 101.9+ KB


In [4]:
# 資料篩選
df1[df1['可借'] <= 3]

,站名,所在區,位置,日期,狀態,總車輛數,可借,可還,經度,緯度
0,捷運科技大樓站,大安區,復興南路二段235號前,2024-06-19 07:42:21,True,28,2,26,121.54360,25.02605
3,和平公園東側,大安區,和平東路二段118巷33號,2024-06-19 07:34:20,True,11,3,8,121.54282,25.02351
10,辛亥路一段30號前,大安區,辛亥路一段30號,2024-06-19 07:40:29,True,8,2,6,121.52982,25.01986
11,和平復興路口西北側,大安區,復興南路二段236號,2024-06-19 07:41:21,True,9,1,8,121.54332,25.02543
14,羅斯福路三段245號前,大安區,羅斯福路三段245號,2024-06-19 07:40:31,True,8,1,7,121.52989,25.01927
...,...,...,...,...,...,...,...,...,...,...
1410,臺大第一活動中心西南側,臺大公館校區,第一學生活動中心東南側,2024-06-19 07:41:21,True,40,2,38,121.53995,25.01761
1417,臺大四號館東北側,臺大公館校區,臺大園藝暨景觀學系北側,2024-06-19 07:28:19,True,10,3,6,121.53781,25.01703
1418,臺大新生教室南側,臺大公館校區,臺大新生教學館南側,2024-06-19 07:41:21,True,18,1,16,121.53846,25.01960
1423,臺大黑森林西側,臺大公館校區,臺大霖澤館南側,2024-06-19 07:39:21,True,20,2,18,121.54347,25.01995


pandas.DataFrame.query
- DataFrame.query(expr, *, inplace=False, **kwargs)[source]
- Query the columns of a DataFrame with a boolean expression.

pandas.DataFrame.shape
- property DataFrame.shape[source]
- Return a tuple representing the dimensionality of the DataFrame.

In [5]:
#檢查 query(全是字串)
import numpy as np
less3_df = df1.query('可借<=3')
less3_df.shape

(335, 10)

In [6]:
# and/ or 皆可用
df2 = df1.query('可借<=3 and 總車輛數<=10')
df2

,站名,所在區,位置,日期,狀態,總車輛數,可借,可還,經度,緯度
10,辛亥路一段30號前,大安區,辛亥路一段30號,2024-06-19 07:40:29,True,8,2,6,121.52982,25.01986
11,和平復興路口西北側,大安區,復興南路二段236號,2024-06-19 07:41:21,True,9,1,8,121.54332,25.02543
14,羅斯福路三段245號前,大安區,羅斯福路三段245號,2024-06-19 07:40:31,True,8,1,7,121.52989,25.01927
15,溫州公園,大安區,羅斯福路三段283巷11號,2024-06-19 07:37:21,True,5,1,4,121.53156,25.01895
17,溫州停車場,大安區,新生南路三段54巷/溫州街口,2024-06-03 14:08:52,False,5,0,5,121.53310,25.02093
18,銘傳國小側門,大安區,羅斯福路四段21號旁天橋下,2024-06-19 07:41:21,True,7,1,6,121.53549,25.01385
55,和平泰順街口,大安區,和平東路一段/泰順街口東南側,2024-06-19 07:42:21,True,9,3,6,121.53171,25.02621
340,士林區農會社子分部,士林區,社子街162號旁,2024-06-19 07:37:21,True,8,2,6,121.50490,25.08912
429,海巡署(南側),文山區,興隆路三段211號對側(公車亭後),2024-06-19 07:37:21,True,10,3,7,121.55949,24.99472
553,仁愛金山路口,中正區,仁愛路二段31號前,2024-06-19 07:25:22,True,9,1,8,121.52801,25.03840


In [7]:
# 外層''  內層要"" 不同的
df3 = df1.query('所在區=="大安區" and 可借==0')
df3

,站名,所在區,位置,日期,狀態,總車輛數,可借,可還,經度,緯度
17,溫州停車場,大安區,新生南路三段54巷/溫州街口,2024-06-03 14:08:52,False,5,0,5,121.53310,25.02093
58,捷運古亭站(5號出口),大安區,和平東路一段19號前,2024-06-19 07:42:15,True,14,0,14,121.52293,25.02715
74,幸安國小(仁愛路),大安區,仁愛路三段22號,2024-06-19 07:41:21,True,16,0,16,121.53523,25.03753
86,捷運信義安和站(2號出口),大安區,信義路四段188號前,2024-06-19 07:42:21,True,24,0,23,121.55171,25.03304
97,捷運忠孝新生站(3號出口),大安區,忠孝東路三段與10巷口,2024-06-19 07:41:16,True,23,0,23,121.53376,25.04189
98,瑠公公園,大安區,忠孝東路四段49巷4弄/大安路口,2024-03-30 08:59:14,False,20,0,20,121.54612,25.04201
104,捷運忠孝復興站(5號出口),大安區,復興南路一段135巷3號(對面),2024-01-14 04:19:10,False,41,0,41,121.54443,25.04262
108,瑞安街208巷,大安區,瑞安街208巷39號對面,2024-06-19 07:39:15,True,15,0,15,121.54053,25.02979
117,敦化南路二段103巷口,大安區,敦化南路二段99號前,2024-06-19 07:42:21,True,24,0,24,121.54904,25.02870
126,忠孝東路三段217巷口,大安區,忠孝東路三段229-1號前,2024-06-06 15:05:16,False,34,0,34,121.54027,25.04176


In [8]:
n = 0   #代入變數
df4 = df1.query('所在區=="大安區" and 可借==@n')
df4

,站名,所在區,位置,日期,狀態,總車輛數,可借,可還,經度,緯度
17,溫州停車場,大安區,新生南路三段54巷/溫州街口,2024-06-03 14:08:52,False,5,0,5,121.53310,25.02093
58,捷運古亭站(5號出口),大安區,和平東路一段19號前,2024-06-19 07:42:15,True,14,0,14,121.52293,25.02715
74,幸安國小(仁愛路),大安區,仁愛路三段22號,2024-06-19 07:41:21,True,16,0,16,121.53523,25.03753
86,捷運信義安和站(2號出口),大安區,信義路四段188號前,2024-06-19 07:42:21,True,24,0,23,121.55171,25.03304
97,捷運忠孝新生站(3號出口),大安區,忠孝東路三段與10巷口,2024-06-19 07:41:16,True,23,0,23,121.53376,25.04189
98,瑠公公園,大安區,忠孝東路四段49巷4弄/大安路口,2024-03-30 08:59:14,False,20,0,20,121.54612,25.04201
104,捷運忠孝復興站(5號出口),大安區,復興南路一段135巷3號(對面),2024-01-14 04:19:10,False,41,0,41,121.54443,25.04262
108,瑞安街208巷,大安區,瑞安街208巷39號對面,2024-06-19 07:39:15,True,15,0,15,121.54053,25.02979
117,敦化南路二段103巷口,大安區,敦化南路二段99號前,2024-06-19 07:42:21,True,24,0,24,121.54904,25.02870
126,忠孝東路三段217巷口,大安區,忠孝東路三段229-1號前,2024-06-06 15:05:16,False,34,0,34,121.54027,25.04176
